In [1]:
import numpy as np 
import yfinance as yf
from scipy.stats import norm
import matplotlib.pyplot as plt
from dateutil import parser
from datetime import date, datetime

## Compute Price From Black Scholes

In [2]:
# Option Price Parameters
strikes = np.array([90, 100, 120, 130])
r = 0.01 
sigma = 0.3 
S = 110
T = 1

In [3]:
def d1(S, K, T, r, sigma):
    
    return (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))

def d2(S, K, T, r, sigma):
    
    return d1(S, K, T, r, sigma) - sigma * np.sqrt(T)

In [4]:
def call_price(S, K, T, r, sigma):
    
    return S * norm.cdf(d1(S, K, T, r, sigma)) - K * np.exp(-r * T) * norm.cdf(d2(S, K, T, r, sigma))

In [5]:
def put_price(S, K, T, r, sigma):
    
    return K * np.exp(-r * T) * norm.cdf(-d2(S, K, T, r, sigma)) - S * norm.cdf(-d1(S, K, T, r, sigma))

In [6]:
# Compute Option Prices 
long_put = put_price(S, strikes[0], T, r, sigma)
short_put = put_price(S, strikes[1], T, r, sigma)

print("Long Put: ", long_put)
print("Short Put: ", short_put)

short_call = call_price(S, strikes[2], T, r, sigma)
long_call = call_price(S, strikes[3], T, r, sigma)

print("Long Call: ", long_call)
print("Short Call: ", short_call)

# Price Spread
iron_condor = short_call + short_put - long_call - long_put

print("Iron Condor Credit: ", iron_condor)

Long Put:  4.200777117002893
Short Put:  7.71626242057004
Long Call:  6.765585201435236
Short Call:  9.681274758005067
Iron Condor Credit:  6.431174860136977


## Compute Price From Monte Carlo

In [26]:
N = 10000
n = 4
dt = 1.0
sigma = 0.3

u = np.exp(sigma * np.sqrt(dt)) 
d = 1 / u 
S0 = 110 
r = 0.01 

K = strikes

q = (np.exp(r * dt) - d) / (u - d) # risk neutral probability
print("Risk Neutral Probability: ", q)
prices = np.zeros(4)

for i in range(len(strikes)):

    U = np.random.binomial(n, q, N) # result of flipping a coin n times, with  N samples of size n
    temp = S0 * u**U * d**(n - U)
    
    if i == 0 or i == 1:
        V = np.exp(-r * dt * n) * np.sum(np.maximum(K[i] - temp, 0)) / N
    else:
        V = np.exp(-r * dt * n) * np.sum(np.maximum(temp - K[i], 0)) / N
        
    prices[i] = V
    
print("Option Prices: ", prices)
iron_condor_mc = prices[1] + prices[2] - prices[0] - prices[3]
print("Iron Condor Price: ", iron_condor_mc)

Risk Neutral Probability:  0.44205912084670673
Option Prices:  [13.73425005 17.96436332 24.04176558 21.86766564]
Iron Condor Price:  6.40421320749784


#### These prices are now very close. 

## Compute Implied Volatility of Iron Condor Spread

#### The initial sigma was chosen with a wide range. 

#### This hould ensure that the implied volatility will exist between those bounds. 

In [8]:
# Compute Average Strike 
avg_strike = np.mean(strikes)
print("Average Strike: ", avg_strike)

Average Strike:  110.0


In [9]:
# Lets Pretend This is ATM Call 
def bisection_ivol(Price, S, K, T, r):
        
    N = 1000
    tol = 1e-5
    count = 0    
    lower_bound = 0.05
    upper_bound = 2.00
        
    while(count < N):
        
        sigma = (lower_bound + upper_bound) / 2
        Price_implied = S * norm.cdf(d1(S, K, T, r, sigma)) - K * np.exp(-r * T) * norm.cdf(d2(S, K, T, r, sigma))
        
        # Market Price - Iterative Price
        diff = Price - Price_implied 
        
        if np.abs(diff) < tol: 
            return sigma
        
        # Iterative Price too High
        if diff < 0: 
            upper_bound = (sigma + upper_bound) / 2
        
        # Iterative Price too Low
        if diff > 0: 
            lower_bound = (sigma + lower_bound) / 2
        
        count += 1
    
    return 0

In [10]:
# Print Parameters
print("Market Price: ", iron_condor)
print("Stock Price: ", S)
print("Strike Price: ", avg_strike)
print("Time to Expiry: ", T)
print("Risk Free Rate: ", r)

Market Price:  6.431174860136977
Stock Price:  110
Strike Price:  110.0
Time to Expiry:  1
Risk Free Rate:  0.01


In [11]:
iron_condor_vol = bisection_ivol(iron_condor, S, avg_strike, T, r)
print("Iron Condor Implied Volatility: ", iron_condor_vol)

Iron Condor Implied Volatility:  0.13448089902711


#### The implied volatility of the spread is cheaper than the individual options. 

## Compute Iron Condor From AMZN

In [12]:
ticker = "AMZN"
AMZN = yf.Ticker(ticker)
exp_date = AMZN.options
print(exp_date)

('2023-02-10', '2023-02-17', '2023-02-24', '2023-03-03', '2023-03-10', '2023-03-17', '2023-03-24', '2023-04-21', '2023-06-16', '2023-07-21', '2023-09-15', '2023-10-20', '2024-01-19', '2024-03-15', '2024-06-21', '2024-09-20', '2025-01-17', '2025-06-20', '2025-12-19')


In [13]:
date_object = parser.parse(exp_date[10])

print(date_object.date()) 
opt = AMZN.option_chain(str(date_object.date()))
calls = opt.calls
puts = opt.puts

2023-09-15


In [14]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AMZN230915C00045000,2023-01-11 19:40:40+00:00,45.0,51.24,58.50,59.05,0.000000,0.000000,6.0,8,0.774172,True,REGULAR,USD
1,AMZN230915C00050000,2023-01-20 17:44:19+00:00,50.0,47.96,53.85,54.35,0.000000,0.000000,2.0,2544,0.723514,True,REGULAR,USD
2,AMZN230915C00052000,2023-02-02 20:55:06+00:00,52.0,62.34,51.95,52.50,0.000000,0.000000,1.0,4247,0.702640,True,REGULAR,USD
3,AMZN230915C00053000,2022-12-12 18:01:09+00:00,53.0,40.00,44.75,45.35,0.000000,0.000000,3.0,215,0.000010,True,REGULAR,USD
4,AMZN230915C00054000,2022-12-12 18:01:29+00:00,54.0,39.20,43.85,44.45,0.000000,0.000000,1.0,440,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,AMZN230915C04600000,2022-06-03 18:12:55+00:00,4600.0,35.03,30.05,39.50,0.649998,1.890627,4.0,25,3.443849,False,REGULAR,USD
205,AMZN230915C04700000,2022-06-02 19:15:53+00:00,4700.0,35.00,26.50,36.00,0.000000,0.000000,3.0,86,3.341127,False,REGULAR,USD
206,AMZN230915C04800000,2022-06-02 16:10:28+00:00,4800.0,27.92,23.55,32.95,0.000000,0.000000,2.0,0,3.253481,False,REGULAR,USD
207,AMZN230915C04900000,2022-06-02 15:23:46+00:00,4900.0,28.00,21.15,30.35,0.000000,0.000000,1.0,113,3.180117,False,REGULAR,USD


In [15]:
puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,AMZN230915P00045000,2023-02-03 20:03:21+00:00,45.0,0.23,0.20,0.24,0.0,0.000000,77.0,1423.0,0.521489,False,REGULAR,USD
1,AMZN230915P00050000,2023-02-06 18:52:42+00:00,50.0,0.35,0.33,0.38,0.0,0.000000,33.0,545.0,0.504888,False,REGULAR,USD
2,AMZN230915P00052000,2023-02-03 17:24:57+00:00,52.0,0.34,0.40,0.46,0.0,0.000000,1.0,917.0,0.497319,False,REGULAR,USD
3,AMZN230915P00053000,2023-02-06 18:11:47+00:00,53.0,0.46,0.44,0.49,0.1,27.777777,1.0,442.0,0.490728,False,REGULAR,USD
4,AMZN230915P00054000,2023-01-25 15:34:42+00:00,54.0,1.04,0.48,0.53,0.0,0.000000,1.0,662.0,0.486089,False,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,AMZN230915P04600000,2022-04-26 17:05:56+00:00,4600.0,1802.80,2368.50,2386.50,0.0,0.000000,8.0,0.0,0.000010,True,REGULAR,USD
211,AMZN230915P04700000,2022-04-26 17:07:50+00:00,4700.0,1898.70,2468.50,2486.50,0.0,0.000000,2.0,0.0,0.000010,True,REGULAR,USD
212,AMZN230915P04800000,2022-03-24 16:09:15+00:00,4800.0,1602.17,1912.95,1929.70,0.0,0.000000,NaN,1.0,0.000010,True,REGULAR,USD
213,AMZN230915P04900000,2022-05-18 14:24:53+00:00,4900.0,2683.70,2444.00,2461.50,0.0,0.000000,2.0,0.0,0.000010,True,REGULAR,USD


In [16]:
today = date.today()

prices = yf.download(ticker, 
                   start='2020-01-01', 
                   end=today, 
                   progress=False, auto_adjust=True)

S = prices['Close'][-1]

In [17]:
print("Stock Price: ", S)

Stock Price:  103.38999938964844


In [18]:
# Find Puts 
L = len(puts["strike"])
put_prices = np.zeros(2)

for i in range(L):
    
    if puts["strike"][i] == 80:
        put_prices[0] = (puts["bid"][i] + puts["ask"][i]) / 2
        
    if puts["strike"][i] == 90:
        put_prices[1] = (puts["bid"][i] + puts["ask"][i]) / 2

In [19]:
# Find Calls 
L = len(calls["strike"])
call_prices = np.zeros(2)

for i in range(L):
    
    if calls["strike"][i] == 110:
        call_prices[0] = (calls["bid"][i] + calls["ask"][i]) / 2
        
    if puts["strike"][i] == 120:
        call_prices[1] = (calls["bid"][i] + calls["ask"][i]) / 2

In [20]:
print("Put Prices: ", put_prices)
print("Call Prices: ", call_prices)

Put Prices:  [3.175 5.625]
Call Prices:  [9.475 5.975]


In [21]:
amzn_iron_condor = put_prices[1] + call_prices[0] - put_prices[0] - call_prices[1]
print("AMZN Iron Condor Credit: ", amzn_iron_condor)

AMZN Iron Condor Credit:  5.950000000000001


In [22]:
amzn_T = ((date_object.date() - today).days / 252)

In [23]:
# Print Parameters 
print("Market Price: ", amzn_iron_condor)
print("Stock Price: ", S)
print("Average Strike: ", 100)
print("Time to Expiry: ", amzn_T)
print("Risk Free Rate: ", r)

Market Price:  5.950000000000001
Stock Price:  103.38999938964844
Average Strike:  100
Time to Expiry:  0.876984126984127
Risk Free Rate:  0.01


In [24]:
amzn_iron_condor_vol = bisection_ivol(amzn_iron_condor, S, 100, amzn_T, r)
print("Iron Condor Implied Volatility: ", amzn_iron_condor_vol)

Iron Condor Implied Volatility:  0.08996643423955367


## Compare to Yahoo Finace Implied Vol 

#### The implied volatility of the iron condor spread is different than the implied volatility of any strike from the option chain. 

#### It is much lower, meaning that the iron condor should not be sold, but should be bought. 

#### The implied volatility is very cheap when compared to individual options. 